# Introduction
This guide gets you started programming in the low-level TensorFlow APIs (TensorFlow Core), showing you how to:

- Manage your own TensorFlow program (a [tf.Graph](https://tensorflow.google.cn/api_docs/python/tf/Graph)) and TensorFlow runtime (a [tf.Session](https://tensorflow.google.cn/api_docs/python/tf/Session)), instead of relying on Estimators to manage them.
- Run TensorFlow operations, using a [tf.Session](https://tensorflow.google.cn/api_docs/python/tf/Session).
- Use high level components ([datasets](https://tensorflow.google.cn/programmers_guide/low_level_intro#datasets), [layers](https://tensorflow.google.cn/programmers_guide/low_level_intro#layers), and [feature_columns](https://tensorflow.google.cn/programmers_guide/low_level_intro#feature_columns)) in this low level environment.
- Build your own training loop, instead of using the one [provided by Estimators](https://tensorflow.google.cn/get_started/premade_estimators).

We recommend using the higher level APIs to build models when possible. Knowing TensorFlow Core is valuable for the following reasons:
- Experimentation and debugging are both more straight forward when you can use low level TensorFlow operations directly.
- It gives you a mental model of how things work internally when using the higher level APIs.

## Setup
Before using this guide, [install TensorFlow](https://tensorflow.google.cn/install/index).

To get the most out of this guide, you should know the following:

- How to program in Python.
- At least a little bit about arrays.
- Ideally, something about machine learning.

Feel free to launch python and follow along with this walkthrough. Run the following lines to set up your Python environment:

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

C:\Users\Administrator\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Tensor Values
The central unit of data in TensorFlow is the **tensor**. A tensor consists of a set of primitive values shaped into an array of any number of dimensions. A tensor's **rank** is its number of dimensions, while its **shape** is a tuple of integers specifying the array's length along each dimension. Here are some examples of tensor values:

In [ ]:
3. # a rank 0 tensor; a scalar with shape [],
[1., 2., 3.] # a rank 1 tensor; a vector with shape [3]
[[1., 2., 3.], [4., 5., 6.]] # a rank 2 tensor; a matrix with shape [2, 3]
[[[1., 2., 3.]], [[7., 8., 9.]]] # a rank 3 tensor with shape [2, 1, 3]

TensorFlow uses numpy arrays to represent tensor **values**.

## TensorFlow Core Walkthrough
You might think of TensorFlow Core programs as consisting of two discrete sections:

1. Building the computational graph (a [tf.Graph](https://tensorflow.google.cn/api_docs/python/tf/Graph)).
2. Running the computational graph (using a [tf.Session](https://tensorflow.google.cn/api_docs/python/tf/Session)).

### Graph
A **computational graph** is a series of TensorFlow operations arranged into a graph. The graph is composed of two types of objects.

- [Operations](https://tensorflow.google.cn/api_docs/python/tf/Operation) (or "ops"): The nodes of the graph. Operations describe calculations that consume and produce tensors.
- [Tensors](https://tensorflow.google.cn/api_docs/python/tf/Tensor): The edges in the graph. These represent the values that will flow through the graph. Most TensorFlow functions return tf.Tensors.

> Important: tf.Tensors do not have values, they are just handles to elements in the computation graph.

Let's build a simple computational graph. The most basic operation is a constant. The Python function that builds the operation takes a tensor value as input. The resulting operation takes no inputs. When run, it outputs the value that was passed to the constructor. We can create two floating point constants a and b as follows:

In [2]:
a = tf.constant(3.0, dtype=tf.float32)
b = tf.constant(4.0) # also tf.float32 implicitly
total = a + b
print(a)
print(b)
print(total)

Tensor("Const:0", shape=(), dtype=float32)
Tensor("Const_1:0", shape=(), dtype=float32)
Tensor("add:0", shape=(), dtype=float32)


Notice that printing the tensors does not output the values 3.0, 4.0, and 7.0 as you might expect. The above statements only build the computation graph. These [tf.Tensor](https://tensorflow.google.cn/api_docs/python/tf/Tensor) objects just represent the results of the operations that will be run.

Each operation in a graph is given a unique name. This name is independent of the names the objects are assigned to in Python. Tensors are named after the operation that produces them followed by an output index, as in "add:0" above.

### TensorBoard
TensorFlow provides a utility called TensorBoard. One of TensorBoard's many capabilities is visualizing a computation graph. You can easily do this with a few simple commands.

First you save the computation graph to a TensorBoard summary file as follows:

In [3]:
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())

This will produce an event file in the current directory with a name in the following format:

In [ ]:
events.out.tfevents.{timestamp}.{hostname}

Now, in a new terminal, launch TensorBoard with the following shell command:

In [ ]:
!powershell -command "tensorboard --logdir ."

Then open TensorBoard's [graphs page](http://localhost:6006/#graphs) in your browser, and you should see a graph similar to the following:

![](https://tensorflow.google.cn/images/getting_started_add.png)

For more about TensorBoard's graph visualization tools see [TensorBoard: Graph Visualization](https://tensorflow.google.cn/programmers_guide/graph_viz).

### Session
To evaluate tensors, instantiate a [tf.Session](https://tensorflow.google.cn/api_docs/python/tf/Session) object, informally known as a **session**. A session encapsulates the state of the TensorFlow runtime, and runs TensorFlow operations. If a [tf.Graph](https://tensorflow.google.cn/api_docs/python/tf/Graph) is like a .py file, a [tf.Session](https://tensorflow.google.cn/api_docs/python/tf/Session) is like the python executable.

The following code creates a [tf.Session](https://tensorflow.google.cn/api_docs/python/tf/Session) object and then invokes its run method to evaluate the total tensor we created above:

In [3]:
sess = tf.Session()
print(sess.run(total))

7.0


When you request the output of a node with Session.run TensorFlow backtracks through the graph and runs all the nodes that provide input to the requested output node. So this prints the expected value of 7.0:

You can pass multiple tensors to [tf.Session.run](https://tensorflow.google.cn/api_docs/python/tf/InteractiveSession#run). The run method transparently handles any combination of tuples or dictionaries, as in the following example:

In [4]:
print(sess.run({'ab':(a, b), 'total':total}))

{'ab': (3.0, 4.0), 'total': 7.0}


which returns the results in a structure of the same layout.

During a call to [tf.Session.run](https://tensorflow.google.cn/api_docs/python/tf/InteractiveSession#run) any [tf.Tensor](https://tensorflow.google.cn/api_docs/python/tf/InteractiveSession#run) only has a single value. For example, the following code calls [tf.random_uniform](https://tensorflow.google.cn/api_docs/python/tf/random_uniform) to produce a [tf.Tensor](https://tensorflow.google.cn/api_docs/python/tf/Tensor) that generates a random 3-element vector (with values in [0,1)):

In [5]:
vec = tf.random_uniform(shape=(3,))
out1 = vec + 1
out2 = vec + 2
print(sess.run(vec))
print(sess.run(vec))
print(sess.run((out1, out2)))

[0.65979254 0.6832545  0.8257053 ]
[0.14101684 0.717504   0.46363175]
(array([1.8645155, 1.0823534, 1.7918994], dtype=float32), array([2.8645155, 2.0823534, 2.7918994], dtype=float32))


The result shows a different random value on each call to run, but a consistent value during a single run (out1 and out2 receive the same random input).

Some TensorFlow functions return tf.Operations instead of tf.Tensors. The result of calling run on an Operation is None. You run an operation to cause a side-effect, not to retrieve a value. Examples of this include the [initialization](https://tensorflow.google.cn/programmers_guide/low_level_intro#Initializing%20Layers), and [training](https://tensorflow.google.cn/programmers_guide/low_level_intro#Training) ops demonstrated later.

### Feeding
As it stands, this graph is not especially interesting because it always produces a constant result. A graph can be parameterized to accept external inputs, known as **placeholder**. A **placeholder** is a promise to provide a value later, like a function argument.

In [6]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x + y

The preceding three lines are a bit like a function in which we define two input parameters (x and y) and then an operation on them. We can evaluate this graph with multiple inputs by using the feed_dict argument of the [run method](https://tensorflow.google.cn/api_docs/python/tf/InteractiveSession#run) to feed concrete values to the placeholders:

In [7]:
print(sess.run(z, feed_dict={x: 3, y: 4.5}))
print(sess.run(z, feed_dict={x: [1, 3], y: [2, 4]}))

7.5
[3. 7.]


Also note that the feed_dict argument can be used to overwrite any tensor in the graph. The only difference between placeholders and other tf.Tensors is that placeholders throw an error if no value is fed to them.

## Datasets
Placeholders work for simple experiments, but [Datasets](https://tensorflow.google.cn/api_docs/python/tf/data) are the preferred method of streaming data into a model.

To get a runnable [tf.Tensor](https://tensorflow.google.cn/api_docs/python/tf/Tensor) from a Dataset you must first convert it to a [tf.data.Iterator](https://tensorflow.google.cn/api_docs/python/tf/data/Iterator), and then call the Iterator's [get_next](https://tensorflow.google.cn/api_docs/python/tf/data/Iterator#get_next) method.

The simplest way to create an Iterator is with the [make_one_shot_iterator](https://tensorflow.google.cn/api_docs/python/tf/data/Dataset#make_one_shot_iterator) method. For example, in the following code the next_item tensor will return a row from the my_data array on each run call:

In [9]:
my_data = [
    [0, 1,],
    [2, 3,],
    [4, 5,],
    [6, 7,],
]
slices = tf.data.Dataset.from_tensor_slices(my_data)
next_item = slices.make_one_shot_iterator().get_next()

Reaching the end of the data stream causes Dataset to throw an [OutOfRangeError](https://tensorflow.google.cn/api_docs/python/tf/errors/OutOfRangeError). For example, the following code reads the next_item until there is no more data to read:

In [10]:
while True:
    try:
        print(sess.run(next_item))
    except tf.errors.OutOfRangeError:
        break

[0 1]
[2 3]
[4 5]
[6 7]


If the Dataset depends on stateful operations you may need to initialize the iterator before using it, as shown below:

In [11]:
r = tf.random_normal([10, 3])
dataset = tf.data.Dataset.from_tensor_slices(r)
iterator = dataset.make_initializable_iterator()
next_row = iterator.get_next()

sess.run(iterator.initializer)
while True:
    try:
        print(sess.run(next_row))
    except tf.errors.OutOfRangeError:
        break

[ 0.7997934   0.21009223 -0.63029647]
[-0.5588665  -0.33276817 -0.7536564 ]
[ 0.28292245 -0.4471136   0.4242405 ]
[ 0.9862797  -0.3605647   0.21552457]
[1.5888007  0.3366526  0.36542574]
[-0.04544698  0.6700727   1.0574926 ]
[ 0.13024512 -1.0903742  -0.28296775]
[-0.53716516  0.26571345  1.4538085 ]
[-0.86609554  0.33431113  0.55967844]
[ 1.8626319 -1.1030481  0.2085993]


For more details on Datasets and Iterators see: [Importing Data](https://tensorflow.google.cn/programmers_guide/datasets).

## Layers
A trainable model must modify the values in the graph to get new outputs with the same input. [Layers](https://tensorflow.google.cn/api_docs/python/tf/layers) are the preferred way to add trainable parameters to a graph.

Layers package together both the variables and the operations that act on them. For example a [densely-connected layer](https://developers.google.cn/machine-learning/glossary/#fully_connected_layer) performs a weighted sum across all inputs for each output and applies an optional [activation function](https://developers.google.cn/machine-learning/glossary/#activation_function). The connection weights and biases are managed by the layer object.

### Creating Layers
The following code creates a [Dense](https://tensorflow.google.cn/api_docs/python/tf/layers/Dense) layer that takes a batch of input vectors, and produces a single output value for each. To apply a layer to an input, call the layer as if it were a function. For example:

In [13]:
x = tf.placeholder(tf.float32, shape=[None, 3])
linear_model = tf.layers.Dense(units=1)
y = linear_model(x)

The layer inspects its input to determine sizes for its internal variables. So here we must set the shape of the x placeholder so that the layer can build a weight matrix of the correct size.

Now that we have defined the calculation of the output, y, there is one more detail we need to take care of before we run the calculation.

### Initializing Layers
The layer contains variables that must be **initialized** before they can be used. While it is possible to initialize variables individually, you can easily initialize all the variables in a TensorFlow graph as follows:

In [14]:
init = tf.global_variables_initializer()
sess.run(init)

> Important: Calling tf.global_variables_initializer only creates and returns a handle to a TensorFlow operation. That op will initialize all the global variables when we run it with tf.Session.run.

Also note that this global_variables_initializer only initializes variables that existed in the graph when the initializer was created. So the initializer should be one of the last things added during graph construction.

### Executing Layers
Now that the layer is initialized, we can evaluate the linear_model's output tensor as we would any other tensor. For example, the following code:

In [15]:
print(sess.run(y, {x: [[1, 2, 3],[4, 5, 6]]}))

[[-1.5315462]
 [-4.045455 ]]


will generate a two-element output vector such as the above.

### Layer Function shortcuts
For each layer class (like [tf.layers.Dense](https://tensorflow.google.cn/api_docs/python/tf/layers/Dense)) TensorFlow also supplies a shortcut function (like [tf.layers.dense](https://tensorflow.google.cn/api_docs/python/tf/layers/dense)). The only difference is that the shortcut function versions create and run the layer in a single call. For example, the following code is equivalent to the earlier version:

In [16]:
x = tf.placeholder(tf.float32, shape=[None, 3])
y = tf.layers.dense(x, units=1)

init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y, {x: [[1, 2, 3], [4, 5, 6]]}))

[[-4.419561]
 [-9.330349]]


While convenient, this approach allows no access to the [tf.layers.Layer](https://tensorflow.google.cn/api_docs/python/tf/layers/Layer) object. This makes introspection and debugging more difficult, and layer reuse impossible.

## Feature columns
The easiest way to experiment with feature columns is using the [tf.feature_column.input_layer](https://tensorflow.google.cn/api_docs/python/tf/feature_column/input_layer) function. This function only accepts [dense columns](https://tensorflow.google.cn/get_started/feature_columns) as inputs, so to view the result of a categorical column you must wrap it in an [tf.feature_column.indicator_column](https://tensorflow.google.cn/api_docs/python/tf/feature_column/indicator_column). For example:

In [17]:
features = {
    'sales' : [[5], [10], [8], [9]],
    'department': ['sports', 'sports', 'gardening', 'gardening']}

department_column = tf.feature_column.categorical_column_with_vocabulary_list(
                'department', ['sports', 'gardening'])
department_column = tf.feature_column.indicator_column(department_column)

columns = [
        tf.feature_column.numeric_column('sales'),
        department_column
]

inputs = tf.feature_column.input_layer(features, columns)

Running the inputs tensor will parse the features into a batch of vectors.

Feature columns can have internal state, like layers, so they often need to be initialized. Categorical columns use [lookup tables](https://tensorflow.google.cn/api_docs/python/tf/contrib/lookup) internally and these require a separate initialization op, [tf.tables_initializer](https://tensorflow.google.cn/api_docs/python/tf/tables_initializer).

In [19]:
var_init = tf.global_variables_initializer()
table_init = tf.tables_initializer()
sess = tf.Session()
sess.run((var_init, table_init))

(None, None)

Once the internal state has been initialized you can run inputs like any other [tf.Tensor](https://tensorflow.google.cn/api_docs/python/tf/Tensor):

In [20]:
print(sess.run(inputs))

[[ 1.  0.  5.]
 [ 1.  0. 10.]
 [ 0.  1.  8.]
 [ 0.  1.  9.]]


Above ouputs shows how the feature columns have packed the input vectors, with the one-hot "department" as the first two indices and "sales" as the third.

## Training
Now that you're familiar with the basics of core TensorFlow, let's train a small regression model manually.

### Define the data
First let's define some inputs, x, and the expected output for each input, y_true:

In [21]:
x = tf.constant([[1], [2], [3], [4]], dtype=tf.float32)
y_true = tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32)

### Define the model
Next, build a simple linear model, with 1 output:

In [22]:
linear_model = tf.layers.Dense(units=1)

y_pred = linear_model(x)

You can evaluate the predictions as follows:

In [23]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y_pred))

[[-0.77852106]
 [-1.5570421 ]
 [-2.3355632 ]
 [-3.1140842 ]]


The model hasn't yet been trained, so the four "predicted" values aren't very good. Here's what we got; your own output will almost certainly differ.

### Loss
To optimize a model, you first need to define the loss. We'll use the mean square error, a standard loss for regression problems.

While you could do this manually with lower level math operations, the tf.losses module provides a set of common loss functions. You can use it to calculate the mean square error as follows:

In [24]:
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)

print(sess.run(loss))

0.26050222


This will produce a loss value, something like above.

### Training
TensorFlow provides [optimizers](https://developers.google.cn/machine-learning/glossary/#optimizer) implementing standard optimization algorithms. These are implemented as sub-classes of [tf.train.Optimizer](https://tensorflow.google.cn/api_docs/python/tf/train/Optimizer). They incrementally change each variable in order to minimize the loss. The simplest optimization algorithm is [gradient descent](https://developers.google.cn/machine-learning/glossary/#gradient_descent), implemented by [tf.train.GradientDescentOptimizer](https://tensorflow.google.cn/api_docs/python/tf/train/GradientDescentOptimizer). It modifies each variable according to the magnitude of the derivative of loss with respect to that variable. For example:

In [25]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

This code builds all the graph components necessary for the optimization, and returns a training operation. When run, the training op will update variables in the graph. You might run it as follows:

In [26]:
for i in range(100):
    _, loss_value = sess.run((train, loss))
    print(loss_value)

0.26050222
0.22732387
0.20402376
0.18757954
0.1758941
0.16751242
0.1614247
0.15693039
0.15354332
0.15092608
0.14884467
0.14713666
0.14568928
0.14442435
0.14328754
0.14224121
0.14125916
0.14032333
0.13942096
0.13854343
0.1376846
0.13684022
0.13600741
0.13518403
0.13436875
0.13356046
0.13275853
0.13196239
0.13117181
0.13038638
0.12960605
0.1288306
0.12805998
0.12729406
0.12653281
0.12577617
0.12502407
0.12427651
0.12353344
0.12279483
0.12206064
0.121330835
0.12060541
0.11988433
0.11916757
0.11845507
0.11774686
0.11704285
0.11634308
0.11564748
0.11495602
0.114268735
0.11358553
0.112906426
0.11223141
0.11156039
0.110893376
0.11023036
0.10957131
0.10891622
0.10826501
0.107617706
0.10697428
0.10633469
0.105698936
0.10506696
0.10443882
0.10381438
0.103193685
0.102576725
0.10196341
0.101353794
0.10074784
0.100145474
0.09954673
0.09895152
0.09835992
0.09777184
0.09718729
0.09660621
0.096028656
0.09545447
0.094883785
0.09431648
0.09375257
0.093192056
0.09263487
0.09208102
0.091530465
0.09098326


Since train is an op, not a tensor, it doesn't return a value when run. To see the progression of the loss during training, we run the loss tensor at the same time, producing output like the above.

### Complete program

In [27]:
x = tf.constant([[1], [2], [3], [4]], dtype=tf.float32)
y_true = tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32)

linear_model = tf.layers.Dense(units=1)

y_pred = linear_model(x)
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)

optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)
for i in range(100):
    _, loss_value = sess.run((train, loss))
    print(loss_value)

print(sess.run(y_pred))

0.16676116
0.16574056
0.16473325
0.16373701
0.16275014
0.16177163
0.16080067
0.15983662
0.15887916
0.157928
0.15698287
0.15604368
0.15511027
0.15418261
0.15326057
0.15234414
0.1514332
0.15052772
0.14962766
0.14873305
0.1478438
0.14695986
0.14608118
0.14520775
0.14433962
0.14347659
0.14261876
0.14176606
0.1409185
0.14007598
0.13923846
0.138406
0.13757849
0.13675591
0.13593829
0.13512552
0.1343176
0.13351457
0.1327163
0.13192281
0.13113406
0.13035002
0.12957066
0.128796
0.12802596
0.12726052
0.12649961
0.1257433
0.12499151
0.12424423
0.12350138
0.12276298
0.12202899
0.12129942
0.120574184
0.11985331
0.11913668
0.1184244
0.11771634
0.11701255
0.116312966
0.11561752
0.11492628
0.114239156
0.11355614
0.112877205
0.11220233
0.11153147
0.110864654
0.110201806
0.10954291
0.10888797
0.10823695
0.10758985
0.10694659
0.106307164
0.10567157
0.10503976
0.10441175
0.103787504
0.10316696
0.10255015
0.10193701
0.101327576
0.10072173
0.10011956
0.09952093
0.09892592
0.09833444
0.09774652
0.09716209
0.0

## Next steps
To learn more about building models with TensorFlow consider the following:

- [Custom Estimators](https://tensorflow.google.cn/get_started/custom_estimators), to learn how to build customized models with TensorFlow. Your knowledge of TensorFlow Core will help you understand and debug your own models.
If you want to learn more about the inner workings of TensorFlow consider the following documents, which go into more depth on many of the topics discussed here:

- [Graphs and Sessions](https://tensorflow.google.cn/programmers_guide/graphs)
- [Tensors](https://tensorflow.google.cn/programmers_guide/tensors)
- [Variables](https://tensorflow.google.cn/programmers_guide/variables)